In [5]:
from utils.config_loader import load_yaml_config, load_path_registry
import os
import sys
import pandas as pd

project_root = "/Users/sravanigunnu/Library/CloudStorage/OneDrive-IBM/Documents/VS_Projects/Complex_Instruction_Following"
sys.path.append(project_root)
# Change the working directory so relative paths work
os.chdir(project_root)
path = load_path_registry()
data_path = load_yaml_config(path["constraint_preprocess"])
final_path = data_path["output_path"]
default_output_path = os.path.join(
    project_root, final_path
)



In [6]:
df = pd.read_csv(final_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   dataset               60 non-null     object
 1   instruction           60 non-null     object
 2   code                  60 non-null     object
 3   test                  20 non-null     object
 4   Characteristics       60 non-null     object
 5   constraints           60 non-null     object
 6   Characteristics_List  60 non-null     object
dtypes: object(7)
memory usage: 3.4+ KB


In [40]:
import pandas as pd

# Load CSV file
df = pd.read_csv("../../data/outputs/constraint_category_data.csv")

# Convert to JSON and save
df.to_json("../../data/outputs/constraint_category_data.json", orient="records", indent=2)


In [41]:
# df = df.drop(columns=["categories_v1","categories_v2"])
# df.to_csv("../../data/outputs/constraint_category_data.json",index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   dataset                    60 non-null     object
 1   instruction                60 non-null     object
 2   code                       60 non-null     object
 3   test                       20 non-null     object
 4   Characteristics            60 non-null     object
 5   constraints                60 non-null     object
 6   Characteristics_List       60 non-null     object
 7   Mapped_Characteristics     60 non-null     object
 8   Mapped_Constraints         60 non-null     object
 9   Mapped_Characteristics_v2  60 non-null     object
 10  Mapped_Constraints_v2      60 non-null     object
 11  categories_v1              60 non-null     object
 12  categories_v2              60 non-null     object
dtypes: object(13)
memory usage: 6.2+ KB


In [74]:
import pandas as pd
import ast
from collections import Counter,defaultdict


# Function to safely evaluate and extract keys from a dictionary-like string
def extract_keys_from_column_v1(column_data,type):
    if type =="dict":   
        keys = []
        for entry in column_data:
            try:
                parsed = ast.literal_eval(entry)
                if isinstance(parsed, dict):
                    keys.extend(parsed.keys())
            except:
                continue
        return keys
    elif type =="list":
        keys =[]
        for entry in column_data:
            result = ast.literal_eval(entry)
            keys = keys + result
        return keys

def extract_keys_from_column(series):
    """Parses each cell into a dictionary, if possible."""
    result = []
    for row in series:
        try:
            parsed = ast.literal_eval(row) if isinstance(row, str) else row
            if isinstance(parsed, dict):
                result.append(parsed)
        except Exception:
            continue
    return result
         
def category_frequency_dict(df, col1, col2, type="dict"):
    keys_characteristics = extract_keys_from_column(df[col1])
    keys_constraints = extract_keys_from_column(df[col2])

    all_keys = keys_characteristics + keys_constraints

    key_counts = Counter()
    example_map = defaultdict(list)

    for row_dict in all_keys:
        for key, vals in row_dict.items():
            key = key.strip()  # normalize key
            key_counts[key] += 1

            if isinstance(vals, list):
                example_map[key].extend(vals[:3 - len(example_map[key])])
                example_map[key] = example_map[key][:3]
            elif isinstance(vals, str):
                if len(example_map[key]) < 3:
                    example_map[key].append(vals)

    # Prepare output
    key_freq_df = pd.DataFrame([
        {"Category": key, "Frequency": freq, "Examples": example_map.get(key, [])}
        for key, freq in key_counts.items()
    ])

    key_freq_df = key_freq_df.sort_values(by="Frequency", ascending=False).reset_index(drop=True)

    return key_freq_df


def category_frequency_list(df,col1,type="list"):
  
    all_keys = extract_keys_from_column_v1(df[col1],type)
    print(all_keys)
    key_counts = Counter(all_keys)
    key_freq_df = pd.DataFrame(key_counts.items(), columns=["Category", "Frequency"])
    key_freq_df = key_freq_df.sort_values(by="Frequency", ascending=False).reset_index(drop=True)

    return key_freq_df

def filter_by_frequency(df,freq):
    filtered_df = df[df["Frequency"] >= freq]
    return filtered_df



In [67]:
col1 = "Mapped_Characteristics"
col2 = "Mapped_Constraints"
freq_df = category_frequency_dict(df,col1,col2)
freq_df.to_csv("../../data/outputs/constraint_categories/mapped_characteristics_v1.csv")
filtered_df = filter_by_frequency(freq_df,freq=3)

# Print result
filtered_df.head()

,Category,Frequency,Examples
0,Error Handling,13,[The program checks for invalid input types (l...
1,Library Usage,7,[Uses math.acos and math.sqrt from Python’s st...
2,Code Structure,6,[Uses any() and comprehensions to check passwo...
3,Input Handling,5,"[Accepts two input lists as arguments., Inputs..."
4,Data Structure,5,"[Defines a 2D NumPy array data with shape (2, ..."


In [73]:
col1 = "Mapped_Characteristics_v2"
col2 = "Mapped_Constraints_v2"
freq_df = category_frequency_dict(df,col1,col2)
freq_df.to_csv("../../data/outputs/constraint_categories/mapped_characteristics_v2.csv")
filtered_df = filter_by_frequency(freq_df,freq=3)

# Print result
print(filtered_df)

               Category  Frequency  \
0        error_handling         19   
1              function         11   
2         documentation         10   
3        data_structure          9   
4         library_usage          8   
5               testing          7   
6              plotting          7   
7         functionality          6   
8                output          6   
9         compatibility          6   
10        output_format          6   
11       initialization          6   
12          performance          6   
13        configuration          4   
14           code_style          4   
15       error handling          4   
16           assertions          3   
17           conversion          3   
18       code_structure          3   
19               method          3   
20            iteration          3   
21              clarity          3   
22               import          3   
23              sorting          3   
24    data_manipulation          3   
25      data

In [75]:
col1 = "categories_v1"
col2 = "categories_v2"
freq_df = category_frequency_list(df,col1,type="list")
freq_df.to_csv("../../data/outputs/constraint_categories/category_generation_v1.csv")
filtered_df = filter_by_frequency(freq_df,freq=3)

# Print result
print(filtered_df)

['Modularity', 'Rounding', 'Vector Calculation', '3D Coordinate System', 'Function Implementation', 'Output Formatting', 'Correctness', 'Simplicity', 'String Manipulation', 'HTML Integration', 'Code Clarity', 'Dynamic Content Generation', 'Conciseness', 'Semantic HTML Usage', 'Recursion', 'Input Validation', 'Error Handling', 'Time Complexity', 'Function Naming', 'Testing', 'Attributes', 'Methods', 'Documentation', 'Error Handling', 'String Formatting', 'Encapsulation', 'Simplicity', 'Function Implementation', 'Text Decryption', 'Random Selection', 'Character Handling', 'Looping and Iteration', 'String Manipulation', 'Preservation of Non-Alphabetic Characters', 'Explanation', 'Code Example', 'Error Handling', 'Output Demonstration', 'Clarity and Accessibility', 'Educational Value', 'Mutation Testing', 'Code Coverage', 'Testing Frameworks', 'Step-by-Step Guide', 'Installation Instructions', 'Code Snippets', 'Reporting', 'Complexity', 'Condition Handling', 'Fee Calculation', 'Maintainabi

In [76]:
freq_df = category_frequency_list(df,col2,type="list")
freq_df.to_csv("../../data/outputs/constraint_categories/category_generation_v2.csv")
filtered_df = filter_by_frequency(freq_df,freq=3)

# Print result
print(filtered_df)

['function', 'output', 'mathematical', 'modularity', 'input/output', 'execution', 'variable', 'string manipulation', 'HTML embedding', 'variable usage', 'functionality demonstration', 'code clarity', 'conciseness', 'modern syntax', 'function', 'error handling', 'recursion', 'input validation', 'time complexity', 'test cases', 'output formatting', 'class', 'method', 'attribute', 'error handling', 'documentation', 'string formatting', 'function', 'control', 'variable', 'loop', 'string', 'random', 'character', 'conditional', 'input/output', 'function', 'error handling', 'example', 'output', 'educational', 'structural control', 'testing', 'mutation', 'coverage', 'installation', 'documentation', 'function', 'module', 'reporting', 'environment', 'assertion', 'function', 'control', 'variable', 'structural', 'complexity', 'mathematical', 'conditional', 'function', 'control', 'variable', 'algorithm', 'output', 'validation', 'loop', 'module', 'formatting', 'function', 'variable', 'control', 'err

In [44]:
df.head()

,dataset,instruction,code,test,Characteristics,constraints,Characteristics_List,Mapped_Characteristics,Mapped_Constraints,Mapped_Characteristics_v2,Mapped_Constraints_v2,categories_v1,categories_v2
0,ajibawa-2023/Python-Code-23k-ShareGPT,Calculate the distance between two points loca...,To calculate the distance between two points i...,NaN,1) Logic is modularized using calculate_distan...,['Implement the distance calculation in a sepa...,['Logic is modularized using calculate_distanc...,{'Modularization': ['Logic is modularized usin...,{'Function Implementation': ['Implement the di...,{'modularization': ['Logic is modularized usin...,{'function': ['Implement the distance calculat...,"[""Modularity"", ""Rounding"", ""Vector Calculation...","[""function"", ""output"", ""mathematical"", ""modula..."
1,ajibawa-2023/Python-Code-23k-ShareGPT,Assuming that the given sentence is stored in ...,"To achieve this, you can use Python's string m...",NaN,1) Embed the HTML tag using an f-string to dyn...,"[""Provide a concise solution using Python's st...",['Embed the HTML tag using an f-string to dyna...,{'HTML Tagging Methodology': ['Embed the HTML ...,"{'Solution Requirements': [""Provide a concise ...",{'string manipulation': ['Embed the HTML tag u...,"{'solution': [""Provide a concise solution usin...","[""String Manipulation"", ""HTML Integration"", ""C...","[""string manipulation"", ""HTML embedding"", ""var..."
2,ajibawa-2023/Python-Code-23k-ShareGPT,Write a program in Python to find the factoria...,Here is a program in Python that finds the fac...,NaN,1) The function factorial is implemented recur...,['The program must use recursion to calculate ...,['The function factorial is implemented recurs...,{'Functionality': ['The function factorial is ...,{'Functionality Requirements': ['The program m...,{'function': ['The function factorial is imple...,{'recursion': ['The program must use recursion...,"[""Recursion"", ""Input Validation"", ""Error Handl...","[""function"", ""error handling"", ""recursion"", ""i..."
3,ajibawa-2023/Python-Code-23k-ShareGPT,Create a Python class for Animal with the foll...,```python\nclass Animal:\n def __init__(sel...,NaN,"The class defines six attributes: species, nam...","['Include detailed docstrings for each method,...","['The class defines six attributes: species, n...",{'Attributes': ['The class defines six attribu...,{'Docstring Requirements': ['Include detailed ...,{'attributes': ['The class defines six attribu...,{'docstring': ['Include detailed docstrings fo...,"[""Attributes"", ""Methods"", ""Documentation"", ""Er...","[""class"", ""method"", ""attribute"", ""error handli..."
4,ajibawa-2023/Python-Code-23k-ShareGPT,How can I decrypt a set of jokes that are encr...,"Sure, here's a Python function that decrypts t...",NaN,The function is named decryptJokes and accepts...,['Provide a complete Python function implement...,['The function is named decryptJokes and accep...,{'Function Definition': ['The function is name...,{'Function Implementation': ['Provide a comple...,{'function': ['The function is named decryptJo...,{'function': ['Provide a complete Python funct...,"[""Function Implementation"", ""Text Decryption"",...","[""function"", ""control"", ""variable"", ""loop"", ""s..."


In [77]:
import pandas as pd
mapped_df = pd.read_csv("../../data/outputs/constraint_categories/mapped_characteristics_v1.csv")
mapped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  451 non-null    int64 
 1   Category    451 non-null    object
 2   Frequency   451 non-null    int64 
 3   Examples    451 non-null    object
dtypes: int64(2), object(2)
memory usage: 14.2+ KB


In [80]:
import pandas as pd
data = pd.read_csv("../../data/outputs/constraint_category_data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   dataset                    60 non-null     object
 1   instruction                60 non-null     object
 2   code                       60 non-null     object
 3   test                       20 non-null     object
 4   Characteristics            60 non-null     object
 5   constraints                60 non-null     object
 6   Characteristics_List       60 non-null     object
 7   Mapped_Characteristics     60 non-null     object
 8   Mapped_Constraints         60 non-null     object
 9   Mapped_Characteristics_v2  60 non-null     object
 10  Mapped_Constraints_v2      60 non-null     object
 11  categories_v1              60 non-null     object
 12  categories_v2              60 non-null     object
dtypes: object(13)
memory usage: 6.2+ KB


In [ ]:
# extract the keys from mapped_chareacteristics column which has a dictionary and create a new column manual_categories
import ast

# Parse the dictionary in each row and extract keys
def extract_keys_from_dict(value):
    try:
        parsed_dict = ast.literal_eval(value)
        if isinstance(parsed_dict, dict):
            return list(parsed_dict.keys())
    except Exception:
        pass
    return []

# Apply the function to extract keys from Mapped_Characteristics
data["Manual_categories"] = data["Mapped_Characteristics"].apply(extract_keys_from_dict)



In [83]:
data["LLM_categories"] = data["Mapped_Constraints"].apply(extract_keys_from_dict)

In [85]:
data["Final_categories"] = data["Manual_categories"]+data["LLM_categories"]

In [86]:
data.head()

,dataset,instruction,code,test,Characteristics,constraints,Characteristics_List,Mapped_Characteristics,Mapped_Constraints,Mapped_Characteristics_v2,Mapped_Constraints_v2,categories_v1,categories_v2,Manual_categories,LLM_categories,Final_categories
0,ajibawa-2023/Python-Code-23k-ShareGPT,Calculate the distance between two points loca...,To calculate the distance between two points i...,NaN,1) Logic is modularized using calculate_distan...,['Implement the distance calculation in a sepa...,['Logic is modularized using calculate_distanc...,{'Modularization': ['Logic is modularized usin...,{'Function Implementation': ['Implement the di...,{'modularization': ['Logic is modularized usin...,{'function': ['Implement the distance calculat...,"[""Modularity"", ""Rounding"", ""Vector Calculation...","[""function"", ""output"", ""mathematical"", ""modula...","[Modularization, Rounding, Vector Calculation,...","[Function Implementation, Output Formatting, M...","[Modularization, Rounding, Vector Calculation,..."
1,ajibawa-2023/Python-Code-23k-ShareGPT,Assuming that the given sentence is stored in ...,"To achieve this, you can use Python's string m...",NaN,1) Embed the HTML tag using an f-string to dyn...,"[""Provide a concise solution using Python's st...",['Embed the HTML tag using an f-string to dyna...,{'HTML Tagging Methodology': ['Embed the HTML ...,"{'Solution Requirements': [""Provide a concise ...",{'string manipulation': ['Embed the HTML tag u...,"{'solution': [""Provide a concise solution usin...","[""String Manipulation"", ""HTML Integration"", ""C...","[""string manipulation"", ""HTML embedding"", ""var...","[HTML Tagging Methodology, String Manipulation...","[Solution Requirements, Code Explanation, Impl...","[HTML Tagging Methodology, String Manipulation..."
2,ajibawa-2023/Python-Code-23k-ShareGPT,Write a program in Python to find the factoria...,Here is a program in Python that finds the fac...,NaN,1) The function factorial is implemented recur...,['The program must use recursion to calculate ...,['The function factorial is implemented recurs...,{'Functionality': ['The function factorial is ...,{'Functionality Requirements': ['The program m...,{'function': ['The function factorial is imple...,{'recursion': ['The program must use recursion...,"[""Recursion"", ""Input Validation"", ""Error Handl...","[""function"", ""error handling"", ""recursion"", ""i...","[Functionality, Input Validation, Error Handli...","[Functionality Requirements, Complexity Requir...","[Functionality, Input Validation, Error Handli..."
3,ajibawa-2023/Python-Code-23k-ShareGPT,Create a Python class for Animal with the foll...,```python\nclass Animal:\n def __init__(sel...,NaN,"The class defines six attributes: species, nam...","['Include detailed docstrings for each method,...","['The class defines six attributes: species, n...",{'Attributes': ['The class defines six attribu...,{'Docstring Requirements': ['Include detailed ...,{'attributes': ['The class defines six attribu...,{'docstring': ['Include detailed docstrings fo...,"[""Attributes"", ""Methods"", ""Documentation"", ""Er...","[""class"", ""method"", ""attribute"", ""error handli...","[Attributes, Constructor, Getter Methods, Sett...","[Docstring Requirements, String Formatting, Me...","[Attributes, Constructor, Getter Methods, Sett..."
4,ajibawa-2023/Python-Code-23k-ShareGPT,How can I decrypt a set of jokes that are encr...,"Sure, here's a Python function that decrypts t...",NaN,The function is named decryptJokes and accepts...,['Provide a complete Python function implement...,['The function is named decryptJokes and accep...,{'Function Definition': ['The function is name...,{'Function Implementation': ['Provide a comple...,{'function': ['The function is named decryptJo...,{'function': ['Provide a complete Python funct...,"[""Function Implementation"", ""Text Decryption"",...","[""function"", ""control"", ""variable"", ""loop"", ""s...","[Function Definition, Random Selection, Decryp...","[Functio

In [87]:
data.to_csv("../../data/outputs/constraint_category_data.csv",index=False)

In [88]:
all_categories = set()

for items in data["Final_categories"]:
    if isinstance(items, list):
        all_categories.update(items)

print(all_categories)


{'Plotting Specifications', 'Sorting Methodology', 'Method Addition', 'Random Selection', 'Database Interaction', 'Error Handling Constraints', 'Library Constraints', 'Functionality and Design', 'Softmax Accumulation', 'INLINE TRANSFORMATION', 'Axis Scaling', 'Function Characteristics', 'Database Table Definition', 'Email Sending', 'Method Parameters', 'String Formatting', 'Non-Alphabetic Character Handling', 'Exclusions', 'DataFrame Structure', 'Code Minimalism', 'Neural Network Configuration', 'Bug Fixes', 'Function Replacement', 'Initialization Optimization', 'Data Mapping', 'Input Requirements', 'Optimization Techniques', 'Naming Conventions', 'Figure Initialization', 'Data Chunking', 'Message Queue Configuration', 'Performance Considerations', 'Case Handling', 'Configuration Enhancement', 'Key Generation', 'Functionality Constraints', 'Contingency Table Construction', 'Plotting Functions', 'Output File Specification', 'Tax Calculation', 'Temporary File Management', 'Functionality'

In [89]:
print(len(all_categories))

451


In [91]:
import json

# Load the JSON file
with open("../../data/outputs/constraint_categories/category_to_group_mapping.json", "r") as f:
    category_to_group = json.load(f)

# Now category_to_group is a Python dictionary
print(list(category_to_group.items())[:10])  # Preview first 10 mappings


[('Error Handling', 'Error Handling and Robustness'), ('Library Usage', 'Library and API Usage'), ('Code Structure', 'Code Structure and Modularity'), ('Input Handling', 'Input and Output Handling'), ('Data Structure', 'Code Structure and Modularity'), ('Documentation', 'Documentation and Readability'), ('Testing', 'Testing and Debugging'), ('Function Definition', 'Code Structure and Modularity'), ('Data Generation', 'Data Processing and Transformation'), ('Code Quality', 'Code Structure and Modularity')]


In [ ]:
import pandas as pd
import openai
import time

# Set your OpenAI API key
openai.api_key = "sk-..."  # Replace with your actual API key

# Load your dataset
df = pd.read_csv("constraint_category_data.csv")  # Replace with your actual file

# Define 14 supercategories
supercategories = [
    "Code Structure and Modularity",
    "Input and Output Handling",
    "Error Handling and Robustness",
    "Data Processing and Transformation",
    "Performance and Optimization",
    "Library and API Usage",
    "Testing and Debugging",
    "Documentation and Readability",
    "Security and Privacy",
    "Reproducibility and Consistency",
    "Mathematical Computation",
    "File and Data Management",
    "UI and Interaction",
    "Domain-Specific Functionality"
]

# Heuristic mapper for supercategories
def assign_supercategories(instruction: str):
    instruction = instruction.lower()
    mapping = []

    if any(k in instruction for k in ["distance", "angle", "sum", "mean", "normalize", "matrix", "calculation"]):
        mapping.append("Mathematical Computation")
    if any(k in instruction for k in ["input", "read", "print", "display", "output", "return"]):
        mapping.append("Input and Output Handling")
    if any(k in instruction for k in ["error", "invalid", "exception", "robust", "check"]):
        mapping.append("Error Handling and Robustness")
    if any(k in instruction for k in ["data", "transform", "reshape", "processing", "convert"]):
        mapping.append("Data Processing and Transformation")
    if any(k in instruction for k in ["efficient", "optimize", "time complexity", "performance"]):
        mapping.append("Performance and Optimization")
    if any(k in instruction for k in ["import", "library", "api", "module"]):
        mapping.append("Library and API Usage")
    if any(k in instruction for k in ["test", "debug", "validate", "assert"]):
        mapping.append("Testing and Debugging")
    if any(k in instruction for k in ["comment", "doc", "explain", "describe"]):
        mapping.append("Documentation and Readability")
    if any(k in instruction for k in ["secure", "encrypt", "password", "authentication"]):
        mapping.append("Security and Privacy")
    if any(k in instruction for k in ["reproduce", "consistent", "deterministic"]):
        mapping.append("Reproducibility and Consistency")
    if any(k in instruction for k in ["file", "save", "csv", "json", "load", "read from"]):
        mapping.append("File and Data Management")
    if any(k in instruction for k in ["ui", "button", "click", "input field", "interface"]):
        mapping.append("UI and Interaction")
    if any(k in instruction for k in ["plot", "email", "web", "html", "flask", "domain"]):
        mapping.append("Domain-Specific Functionality")

    return list(set(mapping))

# Constraint generation using OpenAI API
def generate_constraints(instruction, supercats, verbosity="brief"):
    prompt = f"""You are an assistant that helps generate constraints for a code generation instruction.
Instruction: "{instruction}"
Relevant supercategories: {', '.join(supercats)}
Generate {8 if verbosity == 'verbose' else 2} {verbosity} constraints for this instruction grouped under the given supercategories. Return as a dictionary where each key is a supercategory and the value is a list of constraints."""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",  
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4
        )
        output = response['choices'][0]['message']['content']
        return eval(output.strip())
    except Exception as e:
        print(f"Error generating constraints: {e}")
        return {}

# Process the dataset
supercats_col = []
brief_col = []
verbose_col = []

for idx, row in df.iterrows():
    instruction = row["instruction"]
    cats = assign_supercategories(instruction)
    supercats_col.append(cats)

    brief = generate_constraints(instruction, cats, verbosity="brief")
    time.sleep(1.5)
    verbose = generate_constraints(instruction, cats, verbosity="verbose")
    time.sleep(1.5)

    brief_col.append(brief)
    verbose_col.append(verbose)

# Add to DataFrame
df["Supercategories"] = supercats_col
df["Brief Constraints"] = brief_col
df["Verbose Constraints"] = verbose_col

# Save to CSV
df.to_csv("constraint_augmented_dataset.csv", index=False)
print("✅ Dataset saved as 'constraint_augmented_dataset.csv'")
